In [1]:
from jax import jit, vmap, random
import jax.numpy as np
import numpy as onp
import jax

# enable float 64
from jax.config import config
config.update('jax_enable_x64', True)

In [ ]:
from utils_response import create_df_response
import pandas as pd
import datetime


# How to use the Iterated EAKF

## Define the your process model $f$, your observational model $g$ and the initial guess of the state space $f_0$ 
- See code below
- We use binomial transition between compartments such that the number of people transitioning from compartment X to compartment Y at rate $\tau$ will be:
$$N_{X\rightarrow Y} \sim \text{Binom}\left(X,  1-\exp\left(\tau\cdot \Delta t\right)\right)$$

Note that here $\tau$ could be a function of $Y$ (Force of Infection for example), and that the process model also account for the step size $\Delta t$. In general, the step size can be embedded in the parameters of the model, but as a good practice we keep in as an argument of the process model with defualt to one $\Delta t=1$.

In [ ]:
# https://github.com/google/jax/issues/10144
def binomial_transition(xi, τ, dt=1):
    kb    = np.maximum(1.0 - np.exp(-τ*dt), 0)
    pop   = onp.random.binomial(onp.int64(xi), kb )
    return jax.device_put(pop)

def checkpopvars(x, pop):
    return np.clip(x, 0, pop)

def f(t, x, β, γ, N, dt=1):
    """
    Process model
    """
    S = x.at[0, :].get()
    I = x.at[1, :].get()
    R = x.at[2, :].get()
    C = x.at[3, :].get()


    s2i = binomial_transition(S, β * I / N, dt)
    i2r = binomial_transition(I, γ, dt)

    S  = S - s2i
    I  = I + s2i - i2r
    R  = R + i2r
    C  = s2i

    return np.array([S, I, R, C])

def g(t, x, θ):
    """
    Observational model.
    """
    return x.at[3, :].get()

def f0(pop=1e+6, inf_init=1/100, m =300):
    """
    Initial guess of the state space.
    """
    I0 = pop * 0.01
    S0 = pop - inf_init
    R0 = 0
    C0 = 0

    x0 = np.ones((4, m)) * np.expand_dims(np.array([S0, I0, R0, C0]), -1)

    return x0

## Simulate the model to have one stochastic trajectory for ingest in the Iterated EAKF.

In [ ]:
N = 1000

β_truth = 0.7 # days
γ_truth = 1/7 # days


T    = 70
dt   = 1
ens  = 500
x0    = f0(N, 1/10, m=ens)
x_sim = x0

x_sim = np.full((4, T, ens), np.nan)
x_sim = x_sim.at[:, 0, :].set(x0)

for t in range(1, T):
    x     = f(t, x_sim.at[:, t-1, :].get(), β_truth, γ_truth, N)
    x_sim = x_sim.at[:, t, :].set(x)


C =  x_sim.at[3, :, :].get()

k = jax.random.PRNGKey(1)

obs_use   = C.at[:, onp.random.randint(ens)].get()
obs_use_n = np.maximum(obs_use + np.squeeze(jax.random.normal(k, shape=(T, 1))*0.5), 0)


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax.plot(obs_use_n, label='obs', marker="x")

In [ ]:
observation_df          = pd.DataFrame(obs_use_n, columns=['y1'])
observation_df["oev1"]  = 1 +( 0.2 * observation_df["y1"])**2
observation_df["date"] = pd.date_range(start=datetime.datetime(2020, 1, 1), periods=T, freq='D')
observation_df

In [ ]:
from ifeakf import ifeakf

model_settings ={
    "m": 300,
    "p": 2,
    "k": 1,
    "n": 4,
    "param_name": ["β", "γ"],
    "dates": observation_df["date"].values,
    "param_truth": [β_truth, γ_truth]
    }

if_settings = {
   "Nif"                : 30,
   "type_cooling"       : "geometric",
   "shrinkage_factor"   : 0.9,
   "assimilation_dates" : observation_df["date"].values
}

f_if = lambda t, x, θ: f(t, x, θ.at[0, :].get(), θ.at[1, :].get(), N)
g_if = lambda t, x, θ: g(t, x, θ)

βmin = 0.2
βmax = 1.2

γmin = 1/10
γmax = 1/4

state_space_range = np.array([[0, N], [0, N], [0, N], [0, N]])
parameters_range = np.array([[βmin, βmax], [γmin, γmax]])

σ_perturb = np.array([0.01, 0.001])
θmle, θpost = ifeakf(process_model            = f_if,
                    observational_model       = g_if,
                    state_space_initial_guess = f0,
                    observations_df           = observation_df.set_index("date"),
                    parameters_range          = parameters_range,
                    state_space_range         = state_space_range,
                    model_settings            = model_settings,
                    if_settings               = if_settings,
                    perturbation              = σ_perturb)


In [ ]:
from diagnostic_plot import convergence_plot

p_truth = model_settings["param_truth"]

fig, ax = plt.subplots(2, 1, figsize=(12.5, 7.2), sharex=True)
convergence_plot(θmle, θpost.mean(-1), parameters_range, param_truth=p_truth, ax=ax, fig=fig)